In [ ]:
import os
import shutil
import random
import pickle
import numpy as np
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

In [ ]:
n = 123
c = 77

# 入力/教師
torch.manual_seed(1)
x_torch = torch.normal(0.5,  1., size=(n, c), requires_grad=True)
t_torch = torch.normal(-0.1, 1., size=(n, c), requires_grad=True)
x_bb = bb.FrameBuffer.from_numpy(x_torch.detach().numpy())
t_bb = bb.FrameBuffer.from_numpy(t_torch.detach().numpy())

# Affine
A_torch = nn.Linear(c, c)
A_bb    = bb.DenseAffine([c], input_shape=[c])

weight = A_torch.weight.detach().numpy()
bias   = A_torch.bias.detach().numpy()
A_bb.W().set_numpy(weight)
A_bb.b().set_numpy(bias)

loss_torch = nn.MSELoss(reduction='mean')
loss_bb    = bb.LossMeanSquaredError(reduction="mean")

opt_torch = optim.Adam(A_torch.parameters(), lr=0.001)
#opt_torch = optim.SGD(A_torch.parameters(), lr=0.001)
opt_torch.zero_grad()

opt_bb    = bb.OptimizerAdam(learning_rate=0.001)
#opt_bb    = bb.OptimizerSgd(learning_rate=0.001)
opt_bb.set_variables(A_bb.get_parameters(), A_bb.get_gradients())

criterion_torch = nn.MSELoss()
criterion_bb = bb.LossMeanSquaredError()

y_torch = A_torch(x_torch)
loss_torch = criterion_torch(y_torch, t_torch)
loss_torch.backward()

y_bb = A_bb.forward(x_bb)
dy_bb = criterion_bb.calculate(y_bb, t_bb)
dx_bb = A_bb.backward(dy_bb)

In [ ]:
print(np.std(y_torch.detach().numpy() - y_bb.numpy()))
print(np.max(y_torch.detach().numpy() - y_bb.numpy()))
print(np.min(y_torch.detach().numpy() - y_bb.numpy()))

In [ ]:
print(np.std(A_torch.weight.grad.detach().numpy() - A_bb.dW().numpy()))
print(np.max(A_torch.weight.grad.detach().numpy() - A_bb.dW().numpy()))
print(np.min(A_torch.weight.grad.detach().numpy() - A_bb.dW().numpy()))
print(np.std(A_torch.bias.grad.detach().numpy() - A_bb.db().numpy()))
print(np.max(A_torch.bias.grad.detach().numpy() - A_bb.db().numpy()))
print(np.min(A_torch.bias.grad.detach().numpy() - A_bb.db().numpy()))

In [ ]:
A_torch.weight.grad.detach().numpy()

In [ ]:
A_bb.dW().numpy()